In [1]:
import random
import pandas as pd
import numpy as np
import plotly.express as px
from statistics import mean

In [2]:
class Doctor:
    def __init__(self, name):
        self.name = name
        self.preferences = []
        self.rejected = []
        self.match = None
        self.numProposals = 0
        self.matchRank = None
        self.popularity = None
    
    def SetPreferences(self, hospitals):
        # set each doctors preferences uniformly randomly
        self.preferences = random.sample(hospitals, len(hospitals))
        return None
    
    def SetPreferencesPopularity(self, hospitals):
        # create a copy of hospitals so we don't accidentally modify it
        remainingHospitals = hospitals.copy()

        for _ in range(len(hospitals)):
            # create a list of the remaining hospitals' popularity to use as weights
            popularityList = [h.popularity for h in remainingHospitals]

            # choose a hospital randomly, with the popularities as weights
            nextHospital = random.choices(remainingHospitals, weights = popularityList)[0]

            # add the hospital to the preference list
            self.preferences.append(nextHospital)

            # remove it from the remainingHospitals list
            remainingHospitals.remove(nextHospital)

        return None
    
    def MakeProposal(self):
        self.numProposals += 1

        # Return the highest ranked hospital that the doctor has not already been rejected by
        bestHospital = [x for x in self.preferences if x not in self.rejected][0]
        
        # Add the hospital to rejected
        self.rejected.append(bestHospital)        

        return bestHospital 
    
    def MakeMatch(self, hospital):
        self.match = hospital
        self.matchRank = self.preferences.index(hospital) + 1

    def __repr__(self):
        return f"Doctor {self.name}"
    
class Hospital:
    def __init__(self, name):
        self.name = name
        self.preferences = []
        self.options = []
        self.match = None
        self.matchRank = None
        self.popularity = None
    
    def SetPreferences(self, doctors):
        # set each hospitals preferences uniformly randomly
        self.preferences = random.sample(doctors, len(doctors))
    
    def SetPreferencesPopularity(self, doctors):
        remainingDoctors = doctors.copy()
        for _ in range(len(doctors)):
            popularityList = [d.popularity for d in remainingDoctors]
            nextDoctor = random.choices(remainingDoctors, weights = popularityList)[0]
            self.preferences.append(nextDoctor)
            remainingDoctors.remove(nextDoctor)
        return None

    def ChooseDoctor(self):
        if len(self.options) > 0:
            # Identify Best Doctor in the Existing Options
            index = min([self.preferences.index(doc) for doc in self.options])
            bestDoctor = self.preferences[index]

            # Delete the old match from the doctor's side
            if self.match is not None:
                self.match.match = None
                self.match.matchRank = None

            # Set the new match to the best doctor
            self.match = bestDoctor
            self.matchRank = self.preferences.index(bestDoctor) + 1

            # Keep the bestDoctor in the options list to compare against future proposals
            self.options = [bestDoctor]
        else:
            bestDoctor = None
        return bestDoctor
    
    def __repr__(self):
        return f"Hospital {self.name}"

In [3]:
def DeferredAcceptance(n, doctors, hospitals):
    rounds = 0
    # Make the matched tracker a set to avoid double counting
    matched = set()

    while len(matched) < n:
        ### Each Doctor Makes a Proposal
        for d in doctors:
            if d.match is None:
                h = d.MakeProposal()
                h.options.append(d)

        ### Each Hospital Chooses a Doctor
        for h in hospitals:
            d = h.ChooseDoctor()
            if d is not None:
                d.MakeMatch(h)
                matched.add(h)
        
        rounds += 1
    return rounds

def CheckStability(doctors, hospitals):
    blockingPairs = []
    for d in doctors:
        for h in hospitals:
            # Check if doctor d prefers another hospital h to d's current match and h prefers d to h's current match (or vice versa)
            if d.preferences.index(h) < d.preferences.index(d.match) and h.preferences.index(d) < h.preferences.index(h.match):
                blockingPairs.append([(d, d.match), (h.match, h)])
            elif d.match.preferences.index(h.match) < d.match.preferences.index(d) and h.match.preferences.index(d.match) < h.match.preferences.index(h):
                blockingPairs.append([(d, d.match), (h.match, h)])
    
    return blockingPairs

In [4]:
def SimulateDeferredAcceptance(n, displayMatches = False, usePopularityModel = False):
    doctors = [Doctor(_) for _ in range(n)]
    hospitals = [Hospital(_) for _ in range(n)]

    if usePopularityModel:
        # randomly shuffle the list of doctors to determine the order of popularity
        dRanking = random.sample(doctors, len(doctors))
        hRanking = random.sample(hospitals, len(hospitals))

        # set their popularity value as their index in the random shuffle
        for index, doctor in enumerate(dRanking):
            doctor.popularity = index+1

        for index, hospital in enumerate(hRanking):
            hospital.popularity = (index+1)**2

        for i in range(n):
            doctors[i].SetPreferencesPopularity(hospitals)
            hospitals[i].SetPreferencesPopularity(doctors)
    else:
        for i in range(n):
            doctors[i].SetPreferences(hospitals)
            hospitals[i].SetPreferences(doctors)

    DeferredAcceptance(n, doctors, hospitals)
    if displayMatches:
        for d in doctors:
            display((d, d.match, d.matchRank))

    # check if there are any blocking pairs
    blockingPairs = CheckStability(doctors, hospitals)

    # only return and output if there are no blocking pairs, otherwise throw an error
    if len(blockingPairs) == 0:
        return doctors, hospitals
    else:
        raise AttributeError("Algorithm Output is not a Stable Match")

In [5]:
# n = 10

# doctors = [Doctor(_) for _ in range(n)]
# hospitals = [Hospital(_) for _ in range(n)]

# # randomly shuffle the list of doctors to determine the order of popularity
# dRanking = random.sample(doctors, len(doctors))
# hRanking = random.sample(hospitals, len(hospitals))

# # set their popularity value as their index in the random shuffle
# for index, doctor in enumerate(dRanking):
#     doctor.popularity = index+1

# for index, hospital in enumerate(hRanking):
#     hospital.popularity = index+1

# for i in range(n):
#     doctors[i].SetPreferencesPopularity(hospitals)
#     hospitals[i].SetPreferencesPopularity(doctors)

# for i in range(n):
#     print([(h, h.popularity) for h in doctors[i].preferences])



### Total Number of Proposals

In [6]:
max_n = 200
interval = 25
m = 5

n_values = []
trial = {i+1: [] for i in range(m)}
averageRanks = {i+1: [] for i in range(m)}

for n in range(1, max_n + 2, interval):
    n_values.append(n)
    for i in range(m):
        doctors, hospitals = SimulateDeferredAcceptance(n, usePopularityModel=True)
        
        proposals = [d.numProposals for d in doctors]
        trial[i+1].append(sum(proposals))
        
        doctorRanks = [d.matchRank for d in doctors]
        hospitalRanks = [h.matchRank for h in hospitals]
        averageRanks[i+1].append((mean(doctorRanks), mean(hospitalRanks)))

In [7]:
results_dict = {("Trial " + str(key)): trial[key] for key in trial.keys()}
results_dict["n"] = n_values

totalProposals = pd.DataFrame(results_dict)
totalProposals = totalProposals.set_index("n")

totalProposals["Average"] = totalProposals.mean(axis = 1)
display(totalProposals)

px.line(totalProposals['Average'], title = "Total Proposals vs. Number of Doctors / Hospitals", 
        labels = {"value": "Total Proposals"})

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5,Average
n,,,,,,
1,1,1,1,1,1,1.0
26,131,198,180,182,256,189.4
51,717,631,668,701,770,697.4
76,1669,1459,1600,1413,1282,1484.6
101,2312,2344,1996,2592,2099,2268.6
126,3521,3384,3527,3828,3931,3638.2
151,4472,3909,5565,5084,4762,4758.4
176,7028,5969,6104,5945,7105,6430.2
201,7152,8001,9247,8392,8407,8239.8


In [8]:
results_dict = {("Trial " + str(key)): [t[0] for t in averageRanks[key]] for key in averageRanks.keys()}
results_dict["n"] = n_values

doctorRanksResults = pd.DataFrame(results_dict)
doctorRanksResults = doctorRanksResults.set_index("n").round(2)

results_dict = {("Trial " + str(key)): [t[1] for t in averageRanks[key]] for key in averageRanks.keys()}
results_dict["n"] = n_values

hospitalRanksResults = pd.DataFrame(results_dict)
hospitalRanksResults = hospitalRanksResults.set_index("n").round(2)

### Distribution of the Number of Proposals

In [9]:
n = 500
m = 100
rankThreshold = 0.05

distProposals = []
distHappyDoctors = []
distHappyHospitals = []

for i in range(m):
    doctors, hospitals = SimulateDeferredAcceptance(n, usePopularityModel=True)
    
    proposals = [d.numProposals for d in doctors]
    distProposals.append(sum(proposals))

    happyDoctors = [d for d in doctors if d.matchRank < rankThreshold*n]
    distHappyDoctors.append(len(happyDoctors))

    happyHospitals = [h for h in hospitals if h.matchRank < 4*rankThreshold*n]
    distHappyHospitals.append(len(happyHospitals))

KeyboardInterrupt: 

In [ ]:
px.histogram(pd.Series(distProposals, name = "Proposals"), labels = {"value": "Number of Proposals"})

### Average Rank of Doctors and Hospitals

In [10]:
doctorRanksResults["Average Rank"] = doctorRanksResults.mean(axis = 1)
doctorRanksResults["log n"] = np.log(doctorRanksResults.index)

display(doctorRanksResults)

px.line(doctorRanksResults[['Average Rank', 'log n']], title = "Average Rank of Doctor's Matches vs. Number of Doctors / Hospitals", 
        labels = {"value": "Average Rank of Doctor's Matches"})

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5,Average Rank,log n
n,,,,,,,
1,1.00,1.00,1.00,1.00,1.00,1.000,0.000000
26,5.04,7.62,6.92,7.00,9.85,7.286,3.258097
51,14.06,12.37,13.10,13.75,15.10,13.676,3.931826
76,21.96,19.20,21.05,18.59,16.87,19.534,4.330733
101,22.89,23.21,19.76,25.66,20.78,22.460,4.615121
126,27.94,26.86,27.99,30.38,31.20,28.874,4.836282
151,29.62,25.89,36.85,33.67,31.54,31.514,5.017280
176,39.93,33.91,34.68,33.78,40.37,36.534,5.170484
201,35.58,39.81,46.00,41.75,41.83,40.994,5.303305


In [11]:
hospitalRanksResults["Average Rank"] = hospitalRanksResults.mean(axis = 1)
hospitalRanksResults["n / log n"] = hospitalRanksResults.index / np.log(hospitalRanksResults.index)

display(hospitalRanksResults)

px.line(hospitalRanksResults[['Average Rank', 'n / log n']], title = "Average Rank of Hospital's Matches vs. Number of Doctors / Hospitals",
        labels = {"value": "Average Rank of Hospital's Matches"})

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5,Average Rank,n / log n
n,,,,,,,
1,1.00,1.00,1.00,1.00,1.00,1.000,inf
26,10.85,9.88,10.62,7.73,8.46,9.508,7.980120
51,14.63,15.02,14.84,14.55,12.12,14.232,12.971074
76,15.42,17.96,17.57,19.96,22.93,18.768,17.548991
101,27.07,22.78,23.21,24.18,27.52,24.952,21.884586
126,30.75,29.08,29.40,27.88,25.88,28.598,26.053072
151,36.87,34.68,29.58,33.56,29.97,32.932,30.095989
176,35.39,36.73,37.44,38.91,28.32,35.358,34.039367
201,41.57,37.26,35.71,40.16,41.49,39.238,37.900895


### Distribution of the Ranks

In [ ]:
px.histogram(pd.Series(distHappyDoctors, name = "Happy Doctors"), labels = {"value": "Number of Happy Doctors"})

In [ ]:
px.histogram(pd.Series(distHappyHospitals, name = "Happy Hospitals"), labels = {"value": "Number of Happy Hospitals"})

In [ ]:
# distHappyDoctors